In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
# Import Pandas
import pandas as pd

In [2]:
# Set up the executable path
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/99.0.4844.51/chromedriver_mac64.zip
Driver has been saved in cache [/Users/andreypodvysotskiy/.wdm/drivers/chromedriver/mac64/99.0.4844.51]


In [3]:
# Visits the mars nasa news site
url = 'https://redplanetscience.com'
browser.visit(url)
# Optional delay for loading the page
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [5]:
# Set up the HTML parser
html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

In [6]:
# assign title and summary text to variable
slide_elem.find('div', class_='content_title')

<div class="content_title">NASA's Briefcase-Size MarCO Satellite Picks Up Honors</div>

In [7]:
# Use the parent element to find the first 'a' tag and save it as 'news_title'
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"NASA's Briefcase-Size MarCO Satellite Picks Up Honors"

In [8]:
# Use the parent element to find the paragraph text
news_p = slide_elem.find('div', class_='article_teaser_body').get_text()
news_p

'The twin spacecraft, the first of their kind to fly into deep space, earn a Laureate from Aviation Week & Space Technology.'

### Featured Images

In [9]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [10]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [11]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [12]:
# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars3.jpg'

In [13]:
# Use the base URL to create an absolute URL
img_url = f"https://spaceimages-mars.com/{img_url_rel}"

In [14]:
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns=['description', 'Mars', 'Earth']
df.set_index('description', inplace=True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [15]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

In [16]:
#browser.quit()

# D1: Scrape High-Resolution Mars’ Hemisphere Images and Titles

### Hemispheres

In [53]:
# 1. Use browser to visit the URL 
url = 'https://marshemispheres.com/'

browser.visit(url)

In [54]:
# 2. Create a list to hold the images and titles.
hemisphere_image_urls = []

# 3. Write code to retrieve the image urls and titles for each hemisphere.

# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

# Populate title list
titles = img_soup.find_all('h3')
titles = titles[:-1]

# Hardcoding hemisphere names to find hrefs via partial match
hemisphere_names = ['cerberus', 'schiaparelli', 'syrtis', 'valles']

# Find all possible hemisphere image links 
imgs = img_soup.find_all("a", class_="itemLink product-item")

# Extract href links 
hemi_links = [i.get('href') for i in imgs]

# Reduce to unique set
hemi_links = list(set(hemi_links[:-1]))

images = []
for link in hemi_links:
    #Build full url f-string
    new_url = f"{url}{link}"
    # Visit new url
    browser.visit(new_url)
    # Extract html w/ splinter
    html = browser.html
    # Optional delay 
    browser.is_element_present_by_id('div.wide-image', wait_time=1)
    # Parse html with soup
    img_soup = soup(html, 'html.parser')
    # Find full img url and append to list
    full_img = img_soup.find('img', class_='wide-image')
    images.append(full_img)

hemisphere_image_urls = [{"image": images[i].get('src'), "title": titles[i].text} for i in range(len(images))]

In [55]:
print(hemisphere_image_urls)

[{'image': 'images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg', 'title': 'Cerberus Hemisphere Enhanced'}, {'image': 'images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg', 'title': 'Schiaparelli Hemisphere Enhanced'}, {'image': 'images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg', 'title': 'Syrtis Major Hemisphere Enhanced'}, {'image': 'images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg', 'title': 'Valles Marineris Hemisphere Enhanced'}]


In [56]:
print(len(full_images))

4


In [31]:
print(len(imgs[0]))

1


In [57]:
# 4. Print the list that holds the dictionary of each image url and title.
hemisphere_image_urls

[{'image': 'images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'image': 'images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'image': 'images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'image': 'images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [58]:
# 5. Quit the browser
browser.quit()